In [1]:
from ipywidgets import (RadioButtons,Select,SelectMultiple, VBox, HBox, Layout, Label, Checkbox, Text, IntSlider)
from qgrid import show_grid
label_layout = Layout(width='100%')  # for making sure the labels display correctly
from isatools.create.models import (InterventionStudyDesign, TreatmentFactory, INTERVENTIONS, 
                                   BASE_FACTORS, SampleAssayPlan, IsaModelObjectFactory, TreatmentSequence)
from isatools.model import Investigation
from isatools.isatab import dump_tables_to_dataframes as dumpdf
import ipywidgets as widgets
import qgrid
qgrid.nbinstall(overwrite=True)


### Study design type choice

Please specify if the study is an intervention or an observation.

In [2]:
rad_study_design = Select(options=['Intervention', 'Observation'], value='Intervention', disabled=False)
VBox([Label('Study design type?', layout=label_layout), rad_study_design])

### Intervention study

If specifying an intervention study, please answer the following:
 - Are study subjects exposed to a single intervention or to multiple intervention?
 - Are there 'hard to change' factors, which restrict randomization of experimental unit?
 
*Note: if you chose 'observation' as the study design type, the following choices will be disabled and you should skip to the Observation study section*
 

In [4]:
if rad_study_design.value == 'Intervention':
    study_design = InterventionStudyDesign()
if rad_study_design.value == 'Observation':
    study_design = None
intervention_ui_disabled = not isinstance(study_design, InterventionStudyDesign)
intervention_type = RadioButtons(options=['single', 'multiple'], value='single', disabled=intervention_ui_disabled)
intervention_type_vbox = VBox([Label('Single intervention or to multiple intervention?', layout=label_layout), intervention_type])
free_or_restricted_design = RadioButtons(options=['yes', 'no'], value='no', disabled=intervention_ui_disabled)
free_or_restricted_design_vbox = VBox([Label("Are there 'hard to change' factors?", layout=label_layout), free_or_restricted_design])
#HBox([intervention_type_vbox, free_or_restricted_design_vbox])

In [5]:
hard_to_change_factors_ui_disabled = free_or_restricted_design.value == 'no'
hard_to_change_factors = RadioButtons(options=[1, 2], value=1, disabled=hard_to_change_factors_ui_disabled)
VBox([Label("If applicable, how many 'hard to change factors'?", layout=label_layout), hard_to_change_factors])

In [6]:
repeats = intervention_type.value != 'single'
factorial_design = free_or_restricted_design.value == 'no'
split_plot_design = (free_or_restricted_design.value == 'yes' and hard_to_change_factors.value == 1)
split_split_plot_design = (free_or_restricted_design.value == 'yes' and hard_to_change_factors.value == 2)
print('Interventions: {}'.format('Multiple interventions' if repeats else 'Single intervention'))
design_type = 'factorial'  # always default to factorial
if split_plot_design:
    design_type = 'split plot'
elif split_split_plot_design:
    design_type = 'split split'
print('Design type: {}'.format(design_type))

Interventions: Single intervention
Design type: factorial


#### Factorial design - intervention types

If specifying an factorial design, please list the intervention types here.

In [7]:
factorial_design_ui_disabled = not factorial_design
chemical_intervention = Checkbox(value=True, description='Chemical intervention', disabled=factorial_design_ui_disabled)
behavioural_intervention = Checkbox(value=False, description='Behavioural intervention', disabled=factorial_design_ui_disabled)
surgical_intervention = Checkbox(value=False, description='Surgical intervention', disabled=factorial_design_ui_disabled)
biological_intervention = Checkbox(value=False, description='Biological intervention', disabled=factorial_design_ui_disabled)
radiological_intervention = Checkbox(value=False, description='Radiological intervention', disabled=factorial_design_ui_disabled)
VBox([chemical_intervention, surgical_intervention, biological_intervention, radiological_intervention])

In [8]:
level_uis = []
if chemical_intervention:
    agent_levels = Text(
        value='calpol, no agent',
        placeholder='e.g. cocaine, calpol',
        description='Agent:',
        disabled=False
    )
    dose_levels = Text(
        value='low, high',
        placeholder='e.g. low, high',
        description='Dose levels:',
        disabled=False
    )
    duration_of_exposure_levels = Text(
        value='short, long',
        placeholder='e.g. short, long',
        description='Duration of exposure:',
        disabled=False
    )
VBox([Label("Chemical intervention factor levels:", layout=label_layout), agent_levels, dose_levels, duration_of_exposure_levels])

In [9]:
factory = TreatmentFactory(intervention_type=INTERVENTIONS['CHEMICAL'], factors=BASE_FACTORS)
for agent_level in agent_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[0], agent_level.strip())
for dose_level in dose_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[1], dose_level.strip())
for duration_of_exposure_level in duration_of_exposure_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[2], duration_of_exposure_level.strip())
print('Number of study groups (treatment groups): {}'.format(len(factory.compute_full_factorial_design())))
treatment_sequence = TreatmentSequence(ranked_treatments=factory.compute_full_factorial_design())

Number of study groups (treatment groups): 8


Next, specify if all study groups of the same size, i.e have the same number of subjects? (in other words, are the groups balanced).

In [10]:
group_balanced = RadioButtons(options=['Balanced', 'Unbalanced'], value='Balanced', disabled=False)
VBox([Label('Are study groups balanced?', layout=label_layout), group_balanced])

Provide the number of subject per study group:

In [11]:
group_size = widgets.IntSlider(min=0, max=100, step=1, description='Group size:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')

def on_value_change(change):
    change['new']

group_size.observe(on_value_change, names='value')
display(group_size)

# group_size = IntSlider(min=0, max=100, step=1, description='Group size:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
# group_size


In [12]:
print(group_size.value)
plan = SampleAssayPlan(group_size.value)


18


In [13]:
sample_type = RadioButtons(options=['Blood', 'Sweat', 'Tears', 'Urine'], value='Tears', disabled=False)
VBox([Label('Sample type?', layout=label_layout), sample_type])

How many times each of the samples have been collected?

In [16]:
sampling_size = widgets.IntSlider( min=0, max=20, step=1, description='Sampling size:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')

display(sampling_size)
sampling_size.observe(on_value_change, names='value')

In [17]:

plan.add_sample_type(sample_type.value)
plan.add_sample_plan_record(sample_type.value, sampling_size.value)
print(sampling_size.value)
isa_object_factory = IsaModelObjectFactory(plan, treatment_sequence)

5


In [22]:
# isa_investigation = Investigation(identifier='inv101')
# isa_study = isa_object_factory.create_study_from_plan()
# isa_study.filename = 's_study.txt'
# isa_investigation.studies = [isa_study]
# dataframes = dumpdf(isa_investigation)
# show_grid(next(iter(dataframes.values())))


isa_investigation = Investigation(identifier='inv101')
isa_study = isa_object_factory.create_study_from_plan()
isa_study.filename = 's_study.txt'
isa_investigation.studies = [isa_study]
dataframes = dumpdf(isa_investigation)
sample_table = next(iter(dataframes.values()))
show_grid(sample_table)

In [23]:
print('Total rows generated: {}'.format(len(sample_table)))

Total rows generated: 720
